In [1]:
import os
%pwd

'c:\\Users\\lenovo\\Desktop\\New folder (2)\\CNN_classifier\\research'

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallnacksConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath: Path

In [4]:
from CnnClassifier.constants import *
from CnnClassifier.utils import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallnacksConfig:
        config=self.config.prepare_callbacks
        model_chkpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_chkpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        preapare_callback_config=PrepareCallnacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return preapare_callback_config 


In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

[2023-06-19 10:58:40,876: INFO: utils: NumExpr defaulting to 4 threads.]


In [8]:
class PrepareCallnacks:
    def __init__(self,config: PrepareCallnacksConfig):
    
        self.config=config

    @property
    def create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_log_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_chkpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_chkpt_callbacks(self):
        return {
            self.create_tb_callbacks,
            self._create_chkpt_callbacks
        }
    

In [9]:
try:
    config=ConfigurationManager()
    prepare_callback_config=config.get_prepare_callbacks_config()
    prepare_callback=PrepareCallnacks(config=prepare_callback_config)
    callback_list=prepare_callback.get_tb_chkpt_callbacks()
except  Exception as e:
    raise e

[2023-06-19 11:12:25,530: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-06-19 11:12:25,546: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-06-19 11:12:25,554: INFO: common: create directory  at: artifacts]
[2023-06-19 11:12:25,562: INFO: common: create directory  at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-06-19 11:12:25,562: INFO: common: create directory  at: artifacts\prepare_callbacks\tensorboard_log_dir]
